In [23]:
import os
import selenium.webdriver as webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import csv


user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:97.0) Gecko/20100101 Firefox/97.0'
edge_driver_path = os.path.join(os.getcwd(), 'msedgedriver.exe')
edge_service = Service(edge_driver_path)
edge_options = Options()
edge_options.add_argument(f'user-agent={user_agent}')
browser = webdriver.Edge(service=edge_service, options=edge_options)

browser.get("https://www.bcn.cl/leychile/navegar?idNorma=1160403")
time.sleep(5)

body = browser.find_element("tag name", "body")

ultima_altura = browser.execute_script("return document.body.scrollHeight")

while True:
    body.send_keys(Keys.END)
    time.sleep(2)

    nueva_altura = browser.execute_script("return document.body.scrollHeight")
    if nueva_altura == ultima_altura:
        break
    ultima_altura = nueva_altura

#Una vez la página está estática, usamos beautifulSoup

page_source = browser.page_source
soup = BeautifulSoup(page_source, "html.parser")

Norma = soup.find("div", class_="norma")

Cuerpo_norma = Norma.find("div", class_="cuerpo-norma min")
if Cuerpo_norma is None:
    raise Exception("No se encontró el Cuerpo de la norma. Puede que la estructura de la página haya cambiado.")


tab_content = Cuerpo_norma.find("div", class_="tab-content card contenido-norma")
if tab_content is None:
    raise Exception("No se encontró el tab content. Puede que la estructura de la página haya cambiado.")


contenedor_norma = tab_content.find("div", class_="texto_norma notas-on s-texto-3")
if contenedor_norma is None:
    raise Exception("No se encontró el contenedor de la norma. Puede que la estructura de la página haya cambiado.")

regex_articulo = re.compile(r"Artículo\s+\d+(°)?(?:\s*[A-Za-z]*)?\s*\.-", re.IGNORECASE)

texto_norma = contenedor_norma.find_all("div", class_="pr")
with open("Ley_consumidor.csv",mode="w", newline="", encoding="utf-8") as archivo:
    encabezados = ["Articulo","Texto_articulo"]
    
    writer = csv.DictWriter(archivo, fieldnames=encabezados)
    writer.writeheader()
    articulo_actual = None
    texto_actual = ""
    
    for ley in texto_norma:
        if ley:
            texto = ley.get_text(separator=" ").strip()

            match = regex_articulo.search(texto)
            if match:
                if articulo_actual is not None:
                    writer.writerow({
                        "Articulo": articulo_actual,
                        "Texto_articulo": texto_actual.strip()
                    })
                articulo_actual = match.group().strip()
                texto_actual = texto[len(match.group()):].strip()

            else:
                texto_actual += " " + texto

    if articulo_actual is not None:
         writer.writerow({
                        "Articulo": articulo_actual,
                        "Texto_articulo": texto_actual.strip()
                    })

browser.quit()

In [29]:
import pandas as pd
import csv

df = pd.read_csv("Ley_consumidor.csv", sep=",", quotechar='"', engine="python")
df.head()

,Articulo,Texto_articulo
0,Artículo 1°.-,La presente ley Ley N° 19.496 Art. 1 D.O. 07.0...
1,Artículo 2°.-,Quedan Ley N° 19.496 Art. 2 D.O. 07.03.1997 s...
2,Artículo 2° bis.-,No obstante Ley N° 19.955 Art. único N° 3 D.O...
3,Artículo 2 ter.-,Las Ley 21398 Art. 1 N° 2 D.O. 24.12.2021 nor...
4,Artículo 3°.-,Son Ley N° 19.496 Art. 3 D.O. 07.03.1997 dere...
